Proyecto prueba TW LATAM

Para que este proyecto funcione se debe:

Hacer un Git clone al repositorio https://github.com/bjofres/tw-latam-test-project.git dentro de la carpeta **My Drive\latam-tw-test**

Autorizar la conexión con la cuenta de Google en el drive.mount

El Notebook ipnyb fue desarrollado y testeado en "**Google Colab**"



In [1]:
# Habilita la recarga automática de módulos en Jupyter Notebook.
# Esto mejora el flujo de trabajo de desarrollo al aplicar automáticamente los cambios realizados
# en los módulos importados sin necesidad de reiniciar manualmente el kernel.
%reload_ext autoreload

# Configura el modo de recarga automática.
# La opción "2" asegura que todos los módulos se recarguen automáticamente después de cualquier cambio.
%autoreload 2

# Mensaje de confirmación para indicar que la funcionalidad de autoreload está activa.
print("IPython autoreload activated")

IPython autoreload activated


Mount a content/drive de la cuenta de Google Drive donde almacenaremos el proyecto y la data de Tweets que trabajaremos

In [2]:
# Montar Google Drive
# En este caso no instalamos ninguna librería para Drive puesto estamos utilizando Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cambia el directorio de trabajo a la carpeta especificada en Google Drive.
Esto es útil cuando se trabaja en Google Colab para organizar proyectos y facilitar el acceso a los archivos.

In [3]:
cd /content/drive/My Drive/latam-tw-test/tw-latam-test-project

/content/drive/My Drive/latam-tw-test/tw-latam-test-project


In [4]:
# Librerías necesarias
!pip install -r requirements.txt

Descarga el archivo ZIP y lo extrae para poder realizar las actividades de procesamiento de datos

Si el archivo ya existe no lo descarga nuevamente

In [5]:
import os
import gdown
import zipfile

# Define el directorio donde se guardará el archivo
file_directory = '/content/drive/My Drive/latam-tw-test/tw-latam-test-project/tweets_extracted/'

# Define el nombre del archivo
file_name = 'farmers-protest-tweets-2021-2-4.json'

# Concatenamos la ruta y el nombre del archivo
file_path = os.path.join(file_directory, file_name)
print(f"Ruta completa del archivo: {file_path}")

# ID del archivo en Google Drive para descargar el archivo ZIP
file_id = "1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"

# URL genérica para descargar desde Google Drive
download_url = f"https://drive.google.com/uc?id={file_id}"

# Nombre para el archivo ZIP descargado
zip_output = "tweets.json.zip"

# Verifica si el archivo ya existe
if os.path.exists(file_path):
    print(f"El archivo {file_name} ya existe en {file_directory}. No se descargará nuevamente.")
else:
    # Si el archivo no existe, descarga y descomprime el archivo ZIP
    try:
        # Descargar el archivo ZIP
        print(f"Descargando {zip_output} desde Google Drive...")
        gdown.download(download_url, zip_output, quiet=False)

        # Descomprimir el archivo ZIP
        with zipfile.ZipFile(zip_output, 'r') as zip_ref:
            zip_ref.extractall(file_directory)

        print(f"Archivo descomprimido en la carpeta {file_directory}.")
    except Exception as e:
        print(f"Error al descargar o descomprimir el archivo: {e}")


Ruta completa del archivo: /content/drive/My Drive/latam-tw-test/tw-latam-test-project/tweets_extracted/farmers-protest-tweets-2021-2-4.json
El archivo farmers-protest-tweets-2021-2-4.json ya existe en /content/drive/My Drive/latam-tw-test/tw-latam-test-project/tweets_extracted/. No se descargará nuevamente.


Se importan los modulos python junto con sus funciones  

In [6]:
%run src/q1_memory.py
%run src/q1_time.py
%run src/q2_memory.py
%run src/q2_time.py
%run src/q3_memory.py
%run src/q3_time.py
%run src/utilidades.py

Define las funciones time call y memory_call para modularizar las llamadas de las funciones y las respuesta, evitando la repetición de codigo

In [7]:
# line_profiler y memory_profiler
import line_profiler
from memory_profiler import profile
from memory_profiler import memory_usage

# definición de la función time_call para el registro de tiempos
@time_it
def time_call(function, file_path):
    """
    Perfila el tiempo de ejecución de la función especificada usando line_profiler y devuelve su resultado.

    Args:
        function (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    from line_profiler import LineProfiler

    # Crear una nueva instancia del perfilador para evitar unificar todo el log de hits
    profiler = LineProfiler()

    # Agregar explícitamente la función al perfilador
    profiler.add_function(function)

    # Usar un diccionario para capturar las variables locales
    local_vars = {"file_path": file_path, "result": None, "function": function}

    # Ejecutar y perfilar la función
    profiler.runctx("result = function(file_path)", globals(), local_vars)

    # Mostrar estadísticas del perfilador
    print(f"=== Estadísticas de Line Profiler para {function.__name__} ===")
    profiler.print_stats()
    print("=== Fin de las Estadísticas ===")

    # Retornar el resultado de la función
    return local_vars["result"]


# definición de la función memory_call para el registro de memoria
@time_it
def memory_call(function, file_path):
    """
    Perfila el uso de memoria de la función especificada y devuelve su resultado.

    Args:
        function_name (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    def wrapper():
        return function(file_path)

    # Perfilar memoria usando memory_usage
    mem_usage, result = memory_usage(wrapper, retval=True, interval=0.1)

    # Mostrar estadísticas de memoria
    print("=== Estadísticas de Memory Profiler ===")
    print(f"Uso máximo de memoria: {max(mem_usage):.2f} MiB")
    print(f"Uso inicial de memoria: {mem_usage[0]:.2f} MiB")
    print(f"Uso final de memoria: {mem_usage[-1]:.2f} MiB")
    print("=== Fin de las Estadísticas ===")

    return result

Las top 10 fechas donde hay más tweets más el usuario (username) con más tweets y su cantidad

In [8]:
# Llamada a 10 fechas con mayor cantidad de tweets y el usuario traves de la funcion time_call con la estadistica de tiempo de ejecucion
top_tweets_time = time_call(q1_time, file_path)
print(f"Resultado final: {top_tweets_time}")

=== Estadísticas de Line Profiler para q1_time ===
Timer unit: 1e-09 s

Total time: 23.1944 s
File: /content/drive/My Drive/latam-tw-test/tw-latam-test-project/src/q1_time.py
Function: q1_time at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
     8                                               # Diccionario para almacenar el conteo de tweets por fecha y usuario
     9         1       7055.0   7055.0      0.0      tweet_count_by_date_user = defaultdict(lambda: defaultdict(int))
    10         1        545.0    545.0      0.0      chunksize = 10000  # Leer 10,000 líneas a la vez
    11                                           
    12         1       1007.0   1007.0      0.0      def process_chunk(chunk):
    13                                                   # Convertir la columna 'date' a datetime y extraer 'username'
    14                   

Los top 10 emojis más usados con su respectivo conteo

In [9]:
# Llama a la funcion q2_time a traves de time_call para obtener la estadistica de tiempo en el proceso de busqueda de los top 10 emojis
top_emojis_time_valor = time_call(q2_time, file_path)
print(f"Resultado final: {top_emojis_time_valor}")

=== Estadísticas de Line Profiler para q2_time ===
Timer unit: 1e-09 s

Total time: 20.1339 s
File: /content/drive/My Drive/latam-tw-test/tw-latam-test-project/src/q2_time.py
Function: q2_time at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    13         1      12660.0  12660.0      0.0      emoji_counter = Counter()
    14                                           
    15         2   64190364.0    3e+07      0.3      with open(file_path, 'r', encoding='utf-8') as file:
    16    117408 1791211946.0  15256.3      8.9          for line in file:
    17    117407   29760726.0    253.5      0.1              try:
    18    117407 5588400779.0  47598.5     27.8                  tweet = json.loads(line)  # Cargar cada línea como un objeto JSON
    19    117407   81828323.0    697.0      0.4                  content = tweet.get('content', '')  # Obtener el contenid

El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

In [10]:
# Llama a la funcion q3_time a traves de time_call para obtener la estadistica de tiempo
# Proceso de busqueda de los usuarios mas influyentes
top_mentions_time = time_call(q3_time, file_path)
print(f"Resultado final: {top_mentions_time}")

=== Estadísticas de Line Profiler para q3_time ===
Timer unit: 1e-09 s

Total time: 8.66722 s
File: /content/drive/My Drive/latam-tw-test/tw-latam-test-project/src/q3_time.py
Function: q3_time at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def q3_time(file_path: str) -> List[Tuple[str, int]]:
    14                                               # inicia un collection de contador para menciones
    15         1      11029.0  11029.0      0.0      mention_counter = Counter()
    16                                           
    17                                               # carga tweets del archivo JSON línea por línea
    18         2   69679525.0    3e+07      0.8      with open(file_path, 'r', encoding='utf-8') as file:
    19    117408 1814102691.0  15451.3     20.9          for line in file:
    20    117407   32327526.0    275.3      0.4              try:
    21    117407 5675957347.0  48344.3     65.5

Las top 10 fechas donde hay más tweets más el usuario (username) con más tweets y su cantidad

In [11]:
# Llamada a 10 fechas con mayor cantidad de tweets y el usuario traves de la funcion memory_call con la estadistica de memoria utilizada
top_tweets_memory = memory_call(q1_memory, file_path)
print(f"Resultado final: {top_tweets_memory}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 351.29 MiB
Uso inicial de memoria: 350.76 MiB
Uso final de memoria: 351.29 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 8.6890 segundos
Resultado final: [(datetime.date(2021, 2, 24), 'preetysaini321', 107), (datetime.date(2021, 2, 23), 'Surrypuria', 135), (datetime.date(2021, 2, 22), 'preetysaini321', 110), (datetime.date(2021, 2, 21), 'Surrypuria', 161), (datetime.date(2021, 2, 20), 'MangalJ23056160', 108), (datetime.date(2021, 2, 19), 'Preetm91', 267), (datetime.date(2021, 2, 18), 'neetuanjle_nitu', 195), (datetime.date(2021, 2, 17), 'RaaJVinderkaur', 185), (datetime.date(2021, 2, 16), 'jot__b', 133), (datetime.date(2021, 2, 15), 'jot__b', 134), (datetime.date(2021, 2, 14), 'rebelpacifist', 119), (datetime.date(2021, 2, 13), 'MaanDee08215437', 178), (datetime.date(2021, 2, 12), 'RanbirS00614606', 176)]


Los top 10 emojis más usados con su respectivo conteo

In [12]:
# Llama a la funcion q2_memory a traves de la función memory cal para obtener la estadistica de memoria en el proceso busqueda top emojis
top_emoji_memory = memory_call(q2_memory, file_path)
print(f"Resultado final: {top_emoji_memory}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 351.32 MiB
Uso inicial de memoria: 351.29 MiB
Uso final de memoria: 350.34 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 11.3846 segundos
Resultado final: [('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

In [13]:
# Llama a la funcion q3_memory a traves de la función memory cal para obtener la estadistica de memoria en el proceso busqueda top mentions
top_mentions_time = memory_call(q3_memory, file_path)
print(f"Resultado final: {top_mentions_time}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 350.43 MiB
Uso inicial de memoria: 350.34 MiB
Uso final de memoria: 350.43 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 7.9471 segundos
Resultado final: [('@narendramodi', 2224), ('@Kisanektamorcha', 1829), ('@RakeshTikaitBKU', 1611), ('@PMOIndia', 1395), ('@RahulGandhi', 1106), ('@GretaThunberg', 1007), ('@RaviSinghKA', 1002), ('@UNHumanRights', 958), ('@rihanna', 951), ('@meenaharris', 915)]
